# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./data/train_data_ver3.csv")
test_data = pd.read_csv("./data/test_data_ver3.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'Receip_No_encoded'
]

In [4]:
# train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded',
    'time_gap_All'
]

# test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded',
    'time_gap_All'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 47
train_data_dam DataFrame의 칼럼 수: 27
train_data_autoclave DataFrame의 칼럼 수: 9
train_data_fill1 DataFrame의 칼럼 수: 18
train_data_fill2 DataFrame의 칼럼 수: 17
----test data-----
test_data DataFrame의 칼럼 수: 48
test_data_dam DataFrame의 칼럼 수: 28
test_data_autoclave DataFrame의 칼럼 수: 10
test_data_fill1 DataFrame의 칼럼 수: 19
test_data_fill2 DataFrame의 칼럼 수: 18


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [10]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    
    # 'Normal'과 'AbNormal'을 숫자로 변환
    y_tr = y_tr.map({'Normal': 0, 'AbNormal': 1})
    y_val = y_val.map({'Normal': 0, 'AbNormal': 1})
    
    param = {
        'iterations': trial.suggest_int('iterations', 800, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }

    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),  
    train_data["target"],              
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-09-28 21:24:23,408] A new study created in memory with name: no-name-11b36ed0-cf6d-4f36-80d8-0857c32d7708
[I 2024-09-28 21:24:42,427] Trial 0 finished with value: 0.22614840989399293 and parameters: {'iterations': 1287, 'learning_rate': 0.13203262096748453, 'depth': 7, 'min_data_in_leaf': 4, 'l2_leaf_reg': 4.780689663026719, 'random_strength': 6.824631292225899, 'bagging_temperature': 9.554598960277284, 'border_count': 238, 'scale_pos_weight': 5.61649246971856, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.22614840989399293.
[I 2024-09-28 21:24:56,663] Trial 1 finished with value: 0.22513089005235598 and parameters: {'iterations': 1368, 'learning_rate': 0.0030110113610709835, 'depth': 8, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.0505036749294598, 'random_strength': 5.595100695350062, 'bagging_temperature': 0.8684052021612866, 'border_count': 198, 'scale_pos_weight': 3.8269447050797387, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.22614840989399293.


[I 2024-09-28 21:43:14,552] Trial 19 finished with value: 0.213406292749658 and parameters: {'iterations': 3358, 'learning_rate': 0.049545579171948656, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.0447668625702655, 'random_strength': 8.801924843527562, 'bagging_temperature': 5.107423535375196, 'border_count': 211, 'scale_pos_weight': 3.995902911875188, 'grow_policy': 'SymmetricTree'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 21:43:43,341] Trial 20 finished with value: 0.23109243697478993 and parameters: {'iterations': 1743, 'learning_rate': 0.10635693486578923, 'depth': 6, 'min_data_in_leaf': 3, 'l2_leaf_reg': 3.306712544126854, 'random_strength': 0.07499004497755202, 'bagging_temperature': 7.0639409341162285, 'border_count': 149, 'scale_pos_weight': 6.59149106322385, 'grow_policy': 'Lossguide'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 21:44:21,993] Trial 21 finished with value: 0.24892703862660942 and parameters: {'iterations': 2584,

[I 2024-09-28 21:55:53,558] Trial 38 finished with value: 0.24225352112676057 and parameters: {'iterations': 1318, 'learning_rate': 0.07463018380817037, 'depth': 5, 'min_data_in_leaf': 5, 'l2_leaf_reg': 2.20648430825313, 'random_strength': 0.7099597845819461, 'bagging_temperature': 7.173120086491, 'border_count': 299, 'scale_pos_weight': 1.4957389359717408, 'grow_policy': 'Lossguide'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 21:56:39,752] Trial 39 finished with value: 0.23835616438356164 and parameters: {'iterations': 1510, 'learning_rate': 0.033331346436241444, 'depth': 11, 'min_data_in_leaf': 5, 'l2_leaf_reg': 2.820679591157496, 'random_strength': 2.0640584000918047, 'bagging_temperature': 7.708649257454899, 'border_count': 140, 'scale_pos_weight': 2.7426295772305522, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 21:57:37,356] Trial 40 finished with value: 0.2409972299168975 and parameters: {'iterations': 3672, 'lea

[I 2024-09-28 22:07:10,087] Trial 57 finished with value: 0.2526636225266362 and parameters: {'iterations': 3245, 'learning_rate': 0.018716315248108344, 'depth': 5, 'min_data_in_leaf': 5, 'l2_leaf_reg': 1.2080944891439225, 'random_strength': 6.006384454532739, 'bagging_temperature': 4.494595166489414, 'border_count': 133, 'scale_pos_weight': 1.5064696429659383, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:07:30,405] Trial 58 finished with value: 0.2392523364485981 and parameters: {'iterations': 2471, 'learning_rate': 0.010898504829554186, 'depth': 4, 'min_data_in_leaf': 5, 'l2_leaf_reg': 1.1543792634285646, 'random_strength': 6.835006609366678, 'bagging_temperature': 3.776000964909513, 'border_count': 146, 'scale_pos_weight': 1.0584003459626345, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:08:04,874] Trial 59 finished with value: 0.25588235294117645 and parameters: {'iterations': 2903, '

[I 2024-09-28 22:17:33,302] Trial 76 finished with value: 0.24737945492662475 and parameters: {'iterations': 3042, 'learning_rate': 0.016589349916187636, 'depth': 4, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.3075374737228487, 'random_strength': 5.597890286409766, 'bagging_temperature': 1.8499032124827028, 'border_count': 181, 'scale_pos_weight': 2.839505692834266, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:18:06,325] Trial 77 finished with value: 0.2363405336721728 and parameters: {'iterations': 3375, 'learning_rate': 0.02841046789574591, 'depth': 5, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.0997482536004732, 'random_strength': 4.704250102409622, 'bagging_temperature': 5.14244973243884, 'border_count': 170, 'scale_pos_weight': 2.0203921108814753, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:18:38,177] Trial 78 finished with value: 0.23873325213154692 and parameters: {'iterations': 2654, 'l

[I 2024-09-28 22:27:34,575] Trial 95 finished with value: 0.25921219822109276 and parameters: {'iterations': 3316, 'learning_rate': 0.015606108788933969, 'depth': 5, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.1724587460982485, 'random_strength': 0.6803170313823426, 'bagging_temperature': 2.4689703757026007, 'border_count': 148, 'scale_pos_weight': 2.171669949738932, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:28:14,352] Trial 96 finished with value: 0.25276073619631906 and parameters: {'iterations': 4987, 'learning_rate': 0.014576591391810695, 'depth': 4, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.1858153805835379, 'random_strength': 0.02650579700521229, 'bagging_temperature': 1.8030111122347574, 'border_count': 148, 'scale_pos_weight': 2.203829665616815, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:28:34,863] Trial 97 finished with value: 0.23880597014925375 and parameters: {'iterations': 33

[I 2024-09-28 22:36:56,800] Trial 114 finished with value: 0.2676470588235294 and parameters: {'iterations': 3714, 'learning_rate': 0.042641891791893004, 'depth': 4, 'min_data_in_leaf': 2, 'l2_leaf_reg': 8.305788654098501, 'random_strength': 1.1192880576077218, 'bagging_temperature': 6.40658803595371, 'border_count': 232, 'scale_pos_weight': 1.4289769983412897, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:37:30,603] Trial 115 finished with value: 0.1963927855711423 and parameters: {'iterations': 4186, 'learning_rate': 0.05744433548885574, 'depth': 4, 'min_data_in_leaf': 2, 'l2_leaf_reg': 8.259500452723536, 'random_strength': 1.9748539583742186, 'bagging_temperature': 6.345107720928781, 'border_count': 230, 'scale_pos_weight': 5.787827695291229, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:38:00,349] Trial 116 finished with value: 0.2646616541353384 and parameters: {'iterations': 3686, 'l

[I 2024-09-28 22:48:03,183] Trial 133 finished with value: 0.2656716417910448 and parameters: {'iterations': 4533, 'learning_rate': 0.04037766960453611, 'depth': 4, 'min_data_in_leaf': 1, 'l2_leaf_reg': 8.79194773178056, 'random_strength': 1.9188128204084074, 'bagging_temperature': 7.168564283747498, 'border_count': 214, 'scale_pos_weight': 1.3165401043275868, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:48:38,605] Trial 134 finished with value: 0.24039653035935563 and parameters: {'iterations': 4432, 'learning_rate': 0.05178400339933772, 'depth': 4, 'min_data_in_leaf': 1, 'l2_leaf_reg': 8.776921182832819, 'random_strength': 1.8849464916938148, 'bagging_temperature': 7.434451441797694, 'border_count': 210, 'scale_pos_weight': 1.9444968023846538, 'grow_policy': 'Depthwise'}. Best is trial 17 with value: 0.2723521320495186.
[I 2024-09-28 22:49:21,926] Trial 135 finished with value: 0.25925925925925924 and parameters: {'iterations': 4519, 